<a href="https://colab.research.google.com/github/Praneethkumar57/Deeplearning_projs/blob/main/P_stance_mini_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/train.csv', encoding='latin-1')

In [ ]:
df['Target'].value_counts()

Target
Donald Trump                6362
Joe Biden                   5806
Bernie Sanders              5056
abortion                    2827
death penalty               2627
nuclear energy              2573
gun control                 2404
cloning                     2187
school uniforms             2165
marijuana legalization      1780
minimum wage                1778
fauci                       1464
face masks                  1307
stay at home orders          972
school closures              790
Hillary Clinton              591
Feminist Movement            569
Legalization of Abortion     561
Atheism                      439
Name: count, dtype: int64

In [ ]:
df['Stance'].value_counts()

Stance
AGAINST    15909
FAVOR      13838
NONE       12511
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
df['Target'] = encoder.fit_transform(df['Target'])

In [ ]:
df['Stance'] = encoder.fit_transform(df['Stance'])

In [ ]:
import re

In [ ]:
# removing hashtags from Tweet column because it has no effect on the sentence
def remove_hash(text):
  clean_text = re.sub(r"[#@]\S+",'',text)
  return clean_text

In [ ]:
df['Tweet'] = df['Tweet'].apply(remove_hash)

In [ ]:
# convert the text to lower case

df['Tweet'] = df['Tweet'].apply(lambda x: x.lower())

In [ ]:
df.head()

,Tweet,Target,Stance,ID
0,dear lord thank u for all of ur blessings forg...,0,0,0
1,"blessed are the peacemakers, for they shall be...",0,0,0
2,i am not conformed to this world. i am transfo...,0,0,0
3,salah should be prayed with and warns again...,0,0,0
4,"papa god, i pray that you shower me with more ...",0,0,0


In [ ]:
# remove tags like <head> if any present

def remove_tags(text):
  cleaned_text = re.sub(re.compile('<.*?>'),'',text)
  return cleaned_text

In [ ]:
df['Tweet'] = df['Tweet'].apply(remove_tags)

# Remove stopwords

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sw_list = stopwords.words('english')

In [ ]:
df['Tweet'] = df['Tweet'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [ ]:
df.head()

,Tweet,Target,Stance,ID
0,dear lord thank u ur blessings forgive sins lo...,0,0,0
1,"blessed peacemakers, shall called children god...",0,0,0
2,conformed world. transformed renewing mind.,0,0,0
3,salah prayed warns prayers done surah al-maoon...,0,0,0
4,"papa god, pray shower patience.",0,0,0


In [ ]:
df['Tweet'][0]

'dear lord thank u ur blessings forgive sins lord give strength energy busy day ahead'

In [ ]:
# remove punctuations
def remove_punc(text):
  clean_txt = re.sub(r"[!#$%&'()*+,-./:;<=>?@^\[\]\\\\{|}~]", "", text)
  return clean_txt

In [ ]:
df['Tweet']=df['Tweet'].apply(remove_punc)

# Using Bert model to get embeddings

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer(df['Tweet'].tolist(),padding=True,truncation=True,return_tensors = 'pt')

In [ ]:
tokens

{'input_ids': tensor([[  101,  6203,  2935,  ...,     0,     0,     0],
        [  101, 10190,  3521,  ...,     0,     0,     0],
        [  101, 23758,  2098,  ...,     0,     0,     0],
        ...,
        [  101,  4931,  4364,  ...,     0,     0,     0],
        [  101,  4283,  5094,  ...,     0,     0,     0],
        [  101,  2471,  2066,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
with torch.no_grad():
  outputs = model(**tokens)

In [ ]:
outputs[0]

tensor([[[-2.2788e-01,  3.1381e-02, -2.0654e-01,  ..., -1.2943e-01,
           3.3568e-01,  4.2487e-01],
         [ 7.0551e-02, -1.4657e-01, -3.4243e-01,  ...,  1.4695e-01,
           6.7303e-02, -1.6697e-01],
         [-4.2028e-02, -9.8529e-02, -8.7601e-01,  ..., -6.3229e-01,
          -7.9439e-01,  1.0088e+00],
         ...,
         [-5.0948e-02, -5.0947e-01, -2.9911e-01,  ...,  1.8829e-01,
           1.6041e-01, -1.3439e-01],
         [-5.5577e-02, -5.0979e-01, -2.7210e-01,  ...,  1.8390e-01,
           1.5960e-01, -1.3735e-01],
         [-7.0319e-02, -4.9365e-01, -2.4925e-01,  ...,  1.6894e-01,
           1.2611e-01, -1.0930e-01]],

        [[-4.9288e-01, -6.7161e-02,  1.3222e-01,  ..., -1.0724e-01,
           3.0143e-03,  6.8569e-02],
         [-9.8652e-03, -8.9996e-01,  1.6524e+00,  ...,  8.3078e-02,
          -1.5432e-01, -4.5069e-01],
         [-1.5709e-01, -8.6287e-02,  6.8270e-01,  ...,  3.8608e-01,
          -1.3209e-01, -2.4336e-01],
         ...,
         [-3.4368e-02,  3

In [ ]:
embeddings = outputs.last_hidden_state[:, 0, :].tolist()

In [ ]:
len(outputs.last_hidden_state[:, 0, :])

2179

In [ ]:
df['Tweet'] = embeddings

In [ ]:
df['Tweet'].shape

(2179,)

In [ ]:
final_inputs = []
for i,row in df.iterrows():
  tweet_tensor = torch.tensor(row['Tweet'])
  target_tensor = torch.tensor([row['Target']])
  final_input = torch.cat((tweet_tensor,target_tensor),dim=0)
  final_inputs.append(final_input)

In [ ]:
final_inputs[0]

tensor([-2.2788e-01,  3.1381e-02, -2.0654e-01, -2.0874e-01, -2.4363e-01,
         1.3226e-01,  3.5852e-01,  3.4361e-01, -4.4002e-01, -1.9196e-01,
        -1.1729e-01, -1.6055e-01,  3.0874e-01,  5.3689e-01, -3.0185e-01,
         3.0999e-02, -1.5551e-01,  3.2411e-01,  2.7441e-01, -2.2195e-01,
         1.5810e-01, -3.5067e-01, -1.1615e-01, -4.0580e-02, -3.1469e-01,
        -9.4143e-02,  2.9498e-02, -3.1698e-01, -8.4899e-02,  1.8347e-01,
        -4.5105e-01,  4.6266e-03,  2.6902e-02, -1.4351e-01,  3.5538e-01,
        -4.3721e-01,  3.7118e-01, -1.5707e-01,  1.0862e-03,  3.5599e-01,
         2.2483e-01,  2.3371e-01,  5.0457e-01, -3.5708e-01,  1.2862e-01,
         7.2195e-02, -1.8593e+00,  1.7945e-01, -3.6580e-01, -6.8019e-02,
         1.6810e-01, -2.3384e-01,  3.2307e-01,  1.7184e-01,  1.4592e-01,
         1.6232e-01, -7.9511e-02,  6.1958e-01,  2.1779e-01,  4.0249e-02,
         1.0178e-02,  2.0763e-01, -1.1662e-01, -1.8990e-01,  2.4184e-01,
         1.6126e-01, -3.2245e-02,  4.9884e-01, -1.9

In [ ]:
df['Tweet'] = final_inputs

In [ ]:
df.head(2)

,Tweet,Target,Stance
0,"[tensor(-0.2279), tensor(0.0314), tensor(-0.20...",1,0
1,"[tensor(-0.4929), tensor(-0.0672), tensor(0.13...",1,0


In [ ]:
df = df.drop(columns=['Target'],axis=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
bert_embeddings = torch.stack(df['Tweet'].apply(torch.tensor).tolist())
target = torch.tensor(df['Stance'].tolist())

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:1174: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mapped = lib.map_infer(


In [ ]:
bert_embeddings[0]

tensor([-2.2788e-01,  3.1381e-02, -2.0654e-01, -2.0874e-01, -2.4363e-01,
         1.3226e-01,  3.5852e-01,  3.4361e-01, -4.4002e-01, -1.9196e-01,
        -1.1729e-01, -1.6055e-01,  3.0874e-01,  5.3689e-01, -3.0185e-01,
         3.0999e-02, -1.5551e-01,  3.2411e-01,  2.7441e-01, -2.2195e-01,
         1.5810e-01, -3.5067e-01, -1.1615e-01, -4.0580e-02, -3.1469e-01,
        -9.4143e-02,  2.9498e-02, -3.1698e-01, -8.4899e-02,  1.8347e-01,
        -4.5105e-01,  4.6266e-03,  2.6902e-02, -1.4351e-01,  3.5538e-01,
        -4.3721e-01,  3.7118e-01, -1.5707e-01,  1.0862e-03,  3.5599e-01,
         2.2483e-01,  2.3371e-01,  5.0457e-01, -3.5708e-01,  1.2862e-01,
         7.2195e-02, -1.8593e+00,  1.7945e-01, -3.6580e-01, -6.8019e-02,
         1.6810e-01, -2.3384e-01,  3.2307e-01,  1.7184e-01,  1.4592e-01,
         1.6232e-01, -7.9511e-02,  6.1958e-01,  2.1779e-01,  4.0249e-02,
         1.0178e-02,  2.0763e-01, -1.1662e-01, -1.8990e-01,  2.4184e-01,
         1.6126e-01, -3.2245e-02,  4.9884e-01, -1.9

In [ ]:
dataset = TensorDataset(bert_embeddings, target)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
input_dim = bert_embeddings.size(1)  # Adjust based on the actual size of your BERT embeddings
hidden_dim = 64
output_dim = 2

In [ ]:
model = ClassificationModel(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100  # Adjust as needed
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0
    total_loss = 0.0
    for batch_bert, batch_target in dataloader:
        optimizer.zero_grad()
        output = model(batch_bert)

        # Assuming output is logits for classification
        loss = criterion(output, batch_target)

        loss.backward()
        optimizer.step()
        _, predicted = torch.max(output, 1)
        total_correct += (predicted == batch_target).sum().item()
        total_samples += batch_target.size(0)
        total_loss += loss.item()
    if(epoch+1)%10==0:
        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')
        print(predicted)
        print(batch_target)

Epoch [10/100], Loss: 0.5142, Accuracy: 75.22%
tensor([0, 0, 0])
tensor([0, 0, 0])
Epoch [20/100], Loss: 0.3959, Accuracy: 82.24%
tensor([0, 0, 1])
tensor([0, 0, 1])
Epoch [30/100], Loss: 0.2436, Accuracy: 91.46%
tensor([1, 1, 0])
tensor([1, 1, 0])
Epoch [40/100], Loss: 0.1443, Accuracy: 95.73%
tensor([1, 1, 0])
tensor([1, 1, 0])
Epoch [50/100], Loss: 0.0497, Accuracy: 99.82%
tensor([1, 1, 0])
tensor([1, 1, 0])
Epoch [60/100], Loss: 0.0221, Accuracy: 100.00%
tensor([0, 0, 0])
tensor([0, 0, 0])
Epoch [70/100], Loss: 0.0114, Accuracy: 100.00%
tensor([0, 0, 1])
tensor([0, 0, 1])
Epoch [80/100], Loss: 0.0062, Accuracy: 100.00%
tensor([1, 0, 0])
tensor([1, 0, 0])
Epoch [90/100], Loss: 0.0036, Accuracy: 100.00%
tensor([0, 0, 0])
tensor([0, 0, 0])
Epoch [100/100], Loss: 0.0022, Accuracy: 100.00%
tensor([0, 0, 1])
tensor([0, 0, 1])
